## Step 1: Check GPU & Import Libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image, ImageEnhance
import warnings
warnings.filterwarnings('ignore')

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms, models, datasets
from sklearn.metrics import classification_report, confusion_matrix
import time
import json

# Check GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ PyTorch: {torch.__version__}")
print(f"✅ CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
print(f"✅ Using Device: {device}")

if not torch.cuda.is_available():
    print("\n⚠️  WARNING: GPU not enabled!")
    print("   Go to Runtime → Change runtime type → GPU")

## Step 2: Upload Your Dataset

**Option A:** Upload ZIP file (recommended)
1. Zip your `food_dataset` folder
2. Run the cell below and upload the zip

**Option B:** Mount Google Drive
- If your dataset is already on Google Drive

In [ ]:
# OPTION A: Upload ZIP file
from google.colab import files
import zipfile

print("📤 Please upload your food_dataset.zip file...")
uploaded = files.upload()

# Extract
for filename in uploaded.keys():
    print(f"\n📦 Extracting {filename}...")
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall('/content/')
    print("✅ Extraction complete!")

# Find dataset folder
if os.path.exists('/content/food_dataset'):
    dataset_path = '/content/food_dataset'
elif os.path.exists('/content/food_dataset_augmented'):
    dataset_path = '/content/food_dataset_augmented'
else:
    # List contents to find folder
    print("\n📂 Contents of /content/:")
    for item in os.listdir('/content/'):
        print(f"   {item}")
    dataset_path = input("\nEnter the dataset folder name: ")
    dataset_path = f'/content/{dataset_path}'

print(f"\n✅ Dataset path: {dataset_path}")

In [ ]:
# OPTION B: Mount Google Drive (uncomment if using Drive)
# from google.colab import drive
# drive.mount('/content/drive')
# dataset_path = '/content/drive/MyDrive/food_dataset'  # Change path as needed

## Step 3: Explore Dataset

In [ ]:
print(f"📂 Dataset path: {dataset_path}\n")

# Get categories
categories = [d for d in os.listdir(dataset_path) 
              if os.path.isdir(os.path.join(dataset_path, d))]
categories.sort()

print(f"✅ Found {len(categories)} food categories:\n")

# Count images
data = []
total_images = 0
for cat in categories:
    cat_path = os.path.join(dataset_path, cat)
    images = [f for f in os.listdir(cat_path) 
              if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    count = len(images)
    total_images += count
    data.append({'Category': cat, 'Count': count})
    print(f"  • {cat}: {count} images")

print(f"\n✅ Total: {total_images} images")

# Visualize distribution
df = pd.DataFrame(data).sort_values('Count', ascending=False)
plt.figure(figsize=(14, 5))
plt.bar(range(len(df)), df['Count'], color='steelblue')
plt.xlabel('Food Category')
plt.ylabel('Number of Images')
plt.title('Image Distribution Across Categories')
plt.xticks(range(len(df)), df['Category'], rotation=90)
plt.tight_layout()
plt.show()

## Step 4: Display Sample Images

In [ ]:
# Show samples
n_samples = min(24, len(categories))
fig, axes = plt.subplots(4, 6, figsize=(15, 10))
axes = axes.ravel()

for i, cat in enumerate(categories[:n_samples]):
    cat_path = os.path.join(dataset_path, cat)
    images = [f for f in os.listdir(cat_path) 
              if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    if images:
        img_path = os.path.join(cat_path, images[0])
        img = Image.open(img_path).resize((150, 150))
        axes[i].imshow(img)
        axes[i].set_title(cat, fontsize=8)
        axes[i].axis('off')

for i in range(n_samples, 24):
    axes[i].axis('off')

plt.suptitle('Sample Images from Each Category', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## Step 5: Data Augmentation

Create augmented images to increase dataset size (7x more images).

In [ ]:
from PIL import ImageEnhance, ImageFilter

# Create augmented dataset folder
aug_path = '/content/food_dataset_augmented'
os.makedirs(aug_path, exist_ok=True)

print("🔄 Augmenting images (7x increase)...\n")

original_count = 0
augmented_count = 0

def augment_image(img):
    """Apply augmentations to image"""
    augmented = []
    
    # Original
    augmented.append(img)
    
    # Horizontal flip
    augmented.append(img.transpose(Image.FLIP_LEFT_RIGHT))
    
    # Rotation
    augmented.append(img.rotate(15))
    augmented.append(img.rotate(-15))
    
    # Brightness
    enhancer = ImageEnhance.Brightness(img)
    augmented.append(enhancer.enhance(1.2))
    augmented.append(enhancer.enhance(0.8))
    
    # Contrast
    enhancer = ImageEnhance.Contrast(img)
    augmented.append(enhancer.enhance(1.2))
    
    return augmented

for cat in categories:
    cat_path = os.path.join(dataset_path, cat)
    aug_cat_path = os.path.join(aug_path, cat)
    os.makedirs(aug_cat_path, exist_ok=True)
    
    images = [f for f in os.listdir(cat_path) 
              if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    cat_original = len(images)
    cat_augmented = 0
    
    for img_name in images:
        try:
            img_path = os.path.join(cat_path, img_name)
            img = Image.open(img_path).convert('RGB')
            
            # Get augmented versions
            aug_images = augment_image(img)
            
            # Save all versions
            base_name = os.path.splitext(img_name)[0]
            for j, aug_img in enumerate(aug_images):
                save_path = os.path.join(aug_cat_path, f"{base_name}_aug{j}.jpg")
                aug_img.save(save_path, 'JPEG', quality=90)
                cat_augmented += 1
                
        except Exception as e:
            continue
    
    original_count += cat_original
    augmented_count += cat_augmented
    print(f"  {cat}: {cat_original} → {cat_augmented} images")

print(f"\n{'='*50}")
print(f"📊 AUGMENTATION SUMMARY")
print(f"{'='*50}")
print(f"   Original images: {original_count}")
print(f"   After augmentation: {augmented_count}")
print(f"   Increase: {augmented_count - original_count} new images ({(augmented_count/original_count - 1)*100:.0f}% increase)")
print(f"{'='*50}")

# Update dataset path to use augmented data
dataset_path = aug_path
print(f"\n✅ Using augmented dataset: {dataset_path}")

## Step 6: Prepare Data for Training

In [ ]:
from PIL import Image

IMG_SIZE = 224
BATCH_SIZE = 32

# Data transforms
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# IMPORTANT: Use ORIGINAL dataset for proper train/val split
# (augmented data causes data leakage - same image variants in train AND val)
original_path = '/content/food_dataset'
print(f"📂 Using ORIGINAL dataset to prevent data leakage: {original_path}\n")

# Load dataset without transform first (we'll apply transform in custom datasets)
full_dataset = datasets.ImageFolder(original_path, transform=None)
class_names = full_dataset.classes
num_classes = len(class_names)

# Split 80/20 - this ensures no image overlap between train and val
indices = list(range(len(full_dataset)))
train_size = int(0.8 * len(full_dataset))

# Shuffle with fixed seed for reproducibility
import random
random.seed(42)
random.shuffle(indices)

train_indices = indices[:train_size]
val_indices = indices[train_size:]

# Custom dataset to apply different transforms
class TransformSubset(torch.utils.data.Dataset):
    def __init__(self, dataset, indices, transform):
        self.dataset = dataset
        self.indices = indices
        self.transform = transform
    
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):
        original_idx = self.indices[idx]
        img_path, label = self.dataset.samples[original_idx]
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, label

# Create datasets with proper transforms
train_dataset = TransformSubset(full_dataset, train_indices, train_transform)
val_dataset = TransformSubset(full_dataset, val_indices, val_transform)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print(f"✅ Training samples: {len(train_dataset)}")
print(f"✅ Validation samples: {len(val_dataset)}")
print(f"✅ Number of classes: {num_classes}")
print(f"✅ Batch size: {BATCH_SIZE}")
print(f"✅ Image size: {IMG_SIZE}x{IMG_SIZE}")
print(f"\n⚠️  Using ORIGINAL data (not augmented) to prevent data leakage!")
print(f"📊 Online augmentation applied during training only")
print(f"\n📋 Classes: {class_names[:5]}... (showing first 5)")

## Step 7: Define Deep Learning Models

In [ ]:
def create_model(model_name, num_classes):
    """Create a pre-trained model with custom classifier"""
    
    if model_name == 'ResNet18':
        model = models.resnet18(weights='IMAGENET1K_V1')
        model.fc = nn.Linear(model.fc.in_features, num_classes)
        
    elif model_name == 'ResNet50':
        model = models.resnet50(weights='IMAGENET1K_V1')
        model.fc = nn.Linear(model.fc.in_features, num_classes)
        
    elif model_name == 'EfficientNet-B0':
        model = models.efficientnet_b0(weights='IMAGENET1K_V1')
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
        
    elif model_name == 'EfficientNet-B3':
        model = models.efficientnet_b3(weights='IMAGENET1K_V1')
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
        
    elif model_name == 'DenseNet121':
        model = models.densenet121(weights='IMAGENET1K_V1')
        model.classifier = nn.Linear(model.classifier.in_features, num_classes)
    
    return model

# Models to train
model_names = ['ResNet18', 'ResNet50', 'EfficientNet-B0', 'EfficientNet-B3', 'DenseNet121']

print("📋 Deep Learning Models to train:")
for name in model_names:
    print(f"   • {name}")
print(f"\n✅ All models use ImageNet pre-trained weights (Transfer Learning)")

## Step 8: Training Function

In [ ]:
def train_model(model, train_loader, val_loader, epochs=15, lr=0.001):
    """Train model and return history"""
    
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.5)
    
    history = {'train_acc': [], 'val_acc': [], 'train_loss': [], 'val_loss': []}
    best_acc = 0
    best_model_state = None
    
    for epoch in range(epochs):
        # Training
        model.train()
        train_loss, train_correct, train_total = 0, 0, 0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()
        
        # Validation
        model.eval()
        val_loss, val_correct, val_total = 0, 0, 0
        
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += predicted.eq(labels).sum().item()
        
        train_acc = 100 * train_correct / train_total
        val_acc = 100 * val_correct / val_total
        
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        history['train_loss'].append(train_loss / len(train_loader))
        history['val_loss'].append(val_loss / len(val_loader))
        
        scheduler.step(val_loss / len(val_loader))
        
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_state = model.state_dict().copy()
        
        print(f"  Epoch {epoch+1}/{epochs} - Train: {train_acc:.1f}% - Val: {val_acc:.1f}%")
    
    model.load_state_dict(best_model_state)
    return model, history, best_acc

print("✅ Training function defined!")

## Step 9: Train All Models 🚀

With GPU, this takes about **15-30 minutes** for all 5 models.

In [ ]:
EPOCHS = 15
results = {}
os.makedirs('/content/models', exist_ok=True)

print("🚀 Starting Deep Learning Training...\n")
print(f"Device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"Epochs per model: {EPOCHS}")
print("="*60)

total_start = time.time()

for model_name in model_names:
    print(f"\n📦 Training {model_name}...")
    print("-"*40)
    
    start_time = time.time()
    
    # Create model
    model = create_model(model_name, num_classes)
    
    # Train
    trained_model, history, best_acc = train_model(
        model, train_loader, val_loader, epochs=EPOCHS
    )
    
    train_time = time.time() - start_time
    
    # Store results
    results[model_name] = {
        'model': trained_model,
        'history': history,
        'best_accuracy': best_acc,
        'train_time': train_time
    }
    
    # Save model
    safe_name = model_name.replace('-', '_')
    save_path = f'/content/models/{safe_name}_food.pth'
    torch.save(trained_model.state_dict(), save_path)
    
    print(f"\n  ✅ Best Accuracy: {best_acc:.2f}%")
    print(f"  ⏱️  Time: {train_time/60:.1f} minutes")
    print(f"  💾 Saved: {save_path}")

total_time = time.time() - total_start
print("\n" + "="*60)
print(f"✅ All models trained in {total_time/60:.1f} minutes!")

## Step 10: Compare Model Performance

In [ ]:
# Create comparison dataframe
comparison_data = []
for name, data in results.items():
    comparison_data.append({
        'Model': name,
        'Best Accuracy (%)': data['best_accuracy'],
        'Final Train (%)': data['history']['train_acc'][-1],
        'Final Val (%)': data['history']['val_acc'][-1],
        'Time (min)': data['train_time'] / 60
    })

comparison_df = pd.DataFrame(comparison_data).sort_values('Best Accuracy (%)', ascending=False)

print("\n" + "="*70)
print("📊 MODEL COMPARISON")
print("="*70)
print(comparison_df.to_string(index=False))
print("="*70)

# Find best model - DEFINE THESE VARIABLES HERE
best_model_name = comparison_df.iloc[0]['Model']
best_accuracy = comparison_df.iloc[0]['Best Accuracy (%)']
print(f"\n🏆 BEST MODEL: {best_model_name}")
print(f"   Accuracy: {best_accuracy:.2f}%")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy comparison
colors = ['#2ecc71' if name == best_model_name else '#3498db' for name in comparison_df['Model']]
axes[0].barh(comparison_df['Model'], comparison_df['Best Accuracy (%)'], color=colors)
axes[0].set_xlabel('Accuracy (%)')
axes[0].set_title('Model Comparison - Best Validation Accuracy')
axes[0].set_xlim([0, 100])
for i, v in enumerate(comparison_df['Best Accuracy (%)']):
    axes[0].text(v + 1, i, f'{v:.1f}%', va='center', fontweight='bold')

# Training curves for best model
best_history = results[best_model_name]['history']
axes[1].plot(best_history['train_acc'], label='Train', linewidth=2)
axes[1].plot(best_history['val_acc'], label='Validation', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy (%)')
axes[1].set_title(f'Training History - {best_model_name}')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Step 11: All Models Training Curves

In [ ]:
# Step 11: Classification Report for Best Model
# Get best model
best_model = results[best_model_name]['model']
best_model = best_model.to(device)
best_model.eval()

# Get predictions
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        outputs = best_model(images)
        _, predicted = outputs.max(1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.numpy())

# Classification report
print(f"\n📊 Classification Report for {best_model_name}:\n")
print(classification_report(all_labels, all_preds, target_names=class_names))

In [ ]:
# Step 11b: Confusion Matrix
cm = confusion_matrix(all_labels, all_preds)

plt.figure(figsize=(16, 14))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names)
plt.title(f'Confusion Matrix - {best_model_name} (Accuracy: {best_accuracy:.1f}%)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

## Step 12: Detailed Evaluation of Best Model

In [ ]:
# Step 12: Save Best Model and Results
# Save best model
safe_name = best_model_name.replace('-', '_')
final_model_path = f'/content/best_model_{safe_name}.pth'
torch.save(best_model.state_dict(), final_model_path)
print(f"✅ Best model saved: {final_model_path}")

# Save class names
class_dict = {i: name for i, name in enumerate(class_names)}
with open('/content/class_names.json', 'w') as f:
    json.dump(class_dict, f, indent=2)
print(f"✅ Class names saved")

# Save results summary
summary = {
    'best_model': best_model_name,
    'best_accuracy': float(best_accuracy),
    'num_classes': num_classes,
    'image_size': IMG_SIZE,
    'epochs': EPOCHS,
    'all_results': {name: {
        'accuracy': float(data['best_accuracy']), 
        'time_minutes': float(data['train_time'] / 60)
    } for name, data in results.items()}
}
with open('/content/training_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)
print(f"✅ Training summary saved")

print("\n" + "="*60)
print("🎉 TRAINING COMPLETE!")
print("="*60)
print(f"\n🏆 Best Model: {best_model_name}")
print(f"📈 Best Accuracy: {best_accuracy:.2f}%")

## Step 13: Save All Results

In [ ]:
# Step 13: Package and Download All Models
import shutil

# Create output folder
output_dir = '/content/trained_models'
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir)

# Copy files
shutil.copy(final_model_path, output_dir)
shutil.copy('/content/class_names.json', output_dir)
shutil.copy('/content/training_summary.json', output_dir)

# Copy all model files from /content/models
models_dir = '/content/models'
if os.path.exists(models_dir):
    for f in os.listdir(models_dir):
        shutil.copy(os.path.join(models_dir, f), output_dir)

# Create zip
shutil.make_archive('/content/food_classification_models', 'zip', output_dir)

print("✅ Created: food_classification_models.zip")
print("\n📥 Downloading...")

# Download
from google.colab import files
files.download('/content/food_classification_models.zip')

## Step 14: Download Trained Models